In [1]:
import pandas as pd
import numpy as np
import os
from typing import Optional
from pathlib import Path

In [2]:
def calculate_adi(file_path: str) -> Optional[pd.DataFrame]:
    """
    Tính ADI (Average Demand Interval) cho từng cặp (shop_id, item_id).
    ADI = Tổng số ngày trong khoảng thời gian phân tích / Số ngày có nhu cầu (> 0)
    """
    print("--- Bắt đầu tính toán ADI (Average Demand Interval) ---")
    
    try:
        df_sales = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy tệp tại đường dẫn: {file_path}")
        return None

    # 1. Tiền xử lý dữ liệu
    df_sales['date'] = pd.to_datetime(df_sales['date'], format='%d.%m.%Y')

    # Tính tổng khoảng thời gian phân tích (tính theo ngày)
    min_date = df_sales['date'].min()
    max_date = df_sales['date'].max()
    TOTAL_DAYS_SPAN = (max_date - min_date).days + 1
    
    print(f"Khoảng thời gian phân tích: {TOTAL_DAYS_SPAN} ngày.")

    # 2. Lọc nhu cầu thực tế (loại bỏ hàng trả lại hoặc giao dịch bằng 0)
    df_demand = df_sales[df_sales['item_cnt_day'] > 0].copy()

    # 3. Tính số ngày có nhu cầu (> 0) cho mỗi cặp (shop, item)
    demand_periods = df_demand.groupby(['shop_id', 'item_id'])['date'].nunique().reset_index(name='demand_periods')

    # 4. Tính ADI
    demand_periods['ADI'] = TOTAL_DAYS_SPAN / demand_periods['demand_periods']

    # In kết quả thống kê
    print("\n--- Thống kê ADI (Average Demand Interval) ---")
    print(demand_periods['ADI'].describe())
    
    print("\n--- 5 cặp item/shop có ADI cao nhất (nhu cầu không liên tục) ---")
    top_adi = demand_periods.sort_values(by='ADI', ascending=False).head(5)
    
    # (Đã sửa lỗi, dùng print() thay vì .to_markdown())
    print(top_adi)

    return demand_periods

In [3]:
def calculate_cv2(file_path: str) -> Optional[pd.DataFrame]:
    """
    Tính CV² (Squared Coefficient of Variation) của khối lượng nhu cầu (demand volume)
    cho từng cặp (shop_id, item_id).
    """
    print("--- Bắt đầu tính toán CV² (Squared Coefficient of Variation) ---")
    
    try:
        df_sales = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy tệp tại đường dẫn: {file_path}")
        return None

    # 1. Lọc nhu cầu thực tế
    df_demand = df_sales[df_sales['item_cnt_day'] > 0].copy()

    # 2. Tổng hợp khối lượng nhu cầu hàng ngày
    df_daily_demand = df_demand.groupby(['shop_id', 'item_id', 'date'])['item_cnt_day'].sum().reset_index(name='demand_volume')

    # 3. Tính toán Thống kê
    demand_stats = df_daily_demand.groupby(['shop_id', 'item_id'])['demand_volume'].agg(
        mean_demand=('mean'),
        std_dev_demand=('std')
    ).reset_index()

    # Xử lý trường hợp std_dev là NaN
    demand_stats['std_dev_demand'] = demand_stats['std_dev_demand'].fillna(0)

    # 4. Tính CV và CV²
    demand_stats['CV'] = demand_stats['std_dev_demand'] / demand_stats['mean_demand']
    demand_stats['CV2'] = demand_stats['CV'] ** 2

    # In kết quả thống kê
    print("\n--- Thống kê CV² (Squared Coefficient of Variation) ---")
    print(demand_stats['CV2'].describe())
    
    print("\n--- 5 cặp item/shop có CV² cao nhất (nhu cầu biến động mạnh) ---")
    top_cv2 = demand_stats.sort_values(by='CV2', ascending=False).head(5)
    
    # (Đã sửa lỗi, dùng print() thay vì .to_markdown())
    print(top_cv2)

    return demand_stats

In [4]:
# --- Phần thực thi (tương đương với main.py) ---

# Giả sử tệp notebook (.ipynb) này được lưu trong thư mục 'src/'
# os.getcwd() sẽ trả về thư mục hiện tại (src)
current_dir = Path(os.getcwd())

# Đường dẫn đúng đến tệp sales_train.csv
# (src/data/raw/sales_train.csv)
sales_path = current_dir / "data" / "raw" / "sales_train.csv"

if not sales_path.exists():
    print(f"Lỗi: Không tìm thấy tệp dữ liệu tại: {sales_path}")
    print("Vui lòng đảm bảo tệp 'sales_train.csv' nằm trong 'src/data/raw/'")
else:
    print(f"--- Bắt đầu phân tích dữ liệu từ: {sales_path} ---")

    # --- 1. Chạy Phân tích ADI ---
    # Lưu kết quả vào biến adi_results
    adi_results = calculate_adi(sales_path)
    
    print("\n" + "="*50 + "\n") # Thêm dấu phân cách

    # --- 2. Chạy Phân tích CV² ---
    # Lưu kết quả vào biến cv2_results
    cv2_results = calculate_cv2(sales_path)
    
    print("\n--- Hoàn tất tất cả phân tích ---")

--- Bắt đầu phân tích dữ liệu từ: c:\Users\user\OneDrive\Máy tính\doantt_KHDL\data_science_project\data_science_ecommerce\src\data\raw\sales_train.csv ---
--- Bắt đầu tính toán ADI (Average Demand Interval) ---
Khoảng thời gian phân tích: 1034 ngày.

--- Thống kê ADI (Average Demand Interval) ---
count    424098.000000
mean        496.752362
std         385.693388
min           1.192618
25%         147.714286
50%         344.666667
75%        1034.000000
max        1034.000000
Name: ADI, dtype: float64

--- 5 cặp item/shop có ADI cao nhất (nhu cầu không liên tục) ---
        shop_id  item_id  demand_periods     ADI
212049       30    11789               1  1034.0
298804       44    11801               1  1034.0
99617        17     7501               1  1034.0
363073       53     4388               1  1034.0
363067       53     4378               1  1034.0


--- Bắt đầu tính toán CV² (Squared Coefficient of Variation) ---

--- Thống kê CV² (Squared Coefficient of Variation) ---
count   

In [5]:
# Bạn có thể xem lại kết quả bất cứ lúc nào
if 'adi_results' in locals() and 'cv2_results' in locals():
    print("\n--- Xem 5 dòng đầu của ADI Results ---")
    print(adi_results.head())
    
    print("\n--- Xem 5 dòng đầu của CV² Results ---")
    print(cv2_results.head())


--- Xem 5 dòng đầu của ADI Results ---
   shop_id  item_id  demand_periods         ADI
0        0       30               9  114.888889
1        0       31               7  147.714286
2        0       32              11   94.000000
3        0       33               6  172.333333
4        0       35              12   86.166667

--- Xem 5 dòng đầu của CV² Results ---
   shop_id  item_id  mean_demand  std_dev_demand        CV       CV2
0        0       30     3.444444        2.242271  0.650982  0.423777
1        0       31     1.571429        0.975900  0.621027  0.385675
2        0       32     1.454545        0.522233  0.359035  0.128906
3        0       33     1.000000        0.000000  0.000000  0.000000
4        0       35     1.250000        0.452267  0.361814  0.130909
